In [8]:
import requests
import lxml.html as lh
import pandas as pd

In [9]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [10]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [31]:
tr_elements = doc.xpath('//tr')
col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [32]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    if len(T)!=3:
        break

    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1
        
[len(C) for (title,C) in col]

[181, 181, 181]

### Creating the pandas Df

In [33]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [34]:
#Rearranging Columns
df.columns = ['Borough', 'Neighbourhood','Postcode']
cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]
df = df[cols]
df.head()

,Postcode,Borough,Neighbourhood
0,\n,M1A\n,Not assigned\n
1,\n,M2A\n,Not assigned\n
2,Parkwoods\n,M3A\n,North York\n
3,Victoria Village\n,M4A\n,North York\n
4,"Regent Park, Harbourfront\n",M5A\n,Downtown Toronto\n


### Cleaning the data

In [35]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,,M1A,Not assigned
1,,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto


In [36]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,,M1A,Not assigned
1,,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto
5,"Lawrence Manor, Lawrence Heights",M6A,North York
6,"Queen's Park, Ontario Provincial Government",M7A,Downtown Toronto
7,,M8A,Not assigned
8,Islington Avenue,M9A,Etobicoke
9,"Malvern, Rouge",M1B,Scarborough


### Combining hoods based on similar postcode

In [37]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,,,Canadian postal codes
1,,M1A,Not assigned
2,,M1Y,Not assigned
3,,M1Z,Not assigned
4,,M2A,Not assigned
5,,M2B,Not assigned
6,,M2C,Not assigned
7,,M2E,Not assigned
8,,M2G,Not assigned
9,,M2S,Not assigned


In [38]:
# Removing space
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [39]:
#Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [41]:
# DF Shape
df.shape

(181, 3)

In [42]:
# Saving
df.to_csv(r'df_can.csv')